## Analysis 1

#### Exploratory KG analysis + analysis of pre- and post-literature integration

In this notebook -
1. See graph statistics in Path_Search.ipynb
2. Clustering coefficient
3. Centrality measures
4. Visualization

In [1]:
import os
import os.path
import networkx as nx
import json
import urllib
import traceback
from itertools import islice
from rdflib import Graph, URIRef, BNode, Namespace, Literal
from rdflib.namespace import RDF, OWL
from tqdm import tqdm
import json
import hashlib

In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
#import pheknowlator kg_utils 
import sys
sys.path.append('../')
from pkt_kg.utils import *

In [4]:
KG_PATH = '/home/sanya/PheKnowLatorv2/resources/knowledge_graphs/'
MR_PATH = '/home/sanya/PheKnowLatorv2/machine_read/output_graphs/'
KG_PATH_MERGED = KG_PATH + 'merged_graphs/'
NodeLabelsFile = KG_PATH_MERGED + 'nodeLabels_merged_20220406.pickle'
KG_NAME_MERGED = 'PheKnowLator_machine_read_merged_instance_based_OWLNETS_20220406.gpickle'

In [5]:
with open(NodeLabelsFile, 'rb') as filep:
    nodeLabels = pickle.load(filep)

In [6]:
nx_graph = nx.read_gpickle(KG_PATH_MERGED+KG_NAME_MERGED)

In [7]:
obo = Namespace('http://purl.obolibrary.org/obo/')
napdi = Namespace('http://napdi.org/napdi_srs_imports:')

In [8]:
##read in pickle dictionary with obo identifiers
import pickle
with open(KG_PATH+'strToOBOdict.pickle','rb') as filep:
    obodict = pickle.load(filep)
len(obodict)

40

In [13]:
nx_graph_und = nx_graph.to_undirected()

In [14]:
#Only 1 connected component in graph
components = sorted(list(nx.connected_components(nx_graph_und)), key=len, reverse=True)
cc_content = {x: str(len(components[x])) + ' nodes: ' + ' | '.join(components[x]) if len(components[x]) < 500
              else len(components[x]) for x in range(len(components))}

for k, v in cc_content.items():
    if isinstance(v, int):
        print('COMPONENT: {} Consists of {} nodes'.format(str(k), str(v)))
    else:
        print('\nCOMPONENT: {} Consists of the following nodes:'.format(str(k)))
        for node in v.split(': ')[-1].split(' | '):
            print(node)

COMPONENT: 0 Consists of 745763 nodes


In [17]:
##green tea
subgt = nx_graph.subgraph([
    obodict['greentea'][0],
    obodict['EGCG'][0],
    obodict['EGCG'][1],
    obodict['ECG'][0],
    obodict['epicatechin'][0],
    obodict['catechin'][0],
    obodict['catechin'][1],
    obodict['epigallocatechin'][0],
    obodict['gallocatechin'][0],
    obodict['gallocatechin'][1]
])
get_graph_stats(subgt)

There are 10 nodes, 63 edges, and 5 self-loop(s)
The Average Degree is 6.3
Nodes with highest degree:
Label: catechin
http://purl.obolibrary.org/obo/CHEBI_23053 (degree=34)
Label: (-)-epigallocatechin gallate
http://napdi.org/napdi_srs_imports:epigallocatechin_gallate (degree=26)
Label: Camellia_sinensis_leaf
http://napdi.org/napdi_srs_imports:camellia_sinensis_leaf (degree=18)
Label: (-)-epicatechin
http://purl.obolibrary.org/obo/CHEBI_90 (degree=15)
Label: (-)-epigallocatechin 3-gallate
http://purl.obolibrary.org/obo/CHEBI_4806 (degree=13)
Label: (-)-epicatechin-3-O-gallate
http://purl.obolibrary.org/obo/CHEBI_70255 (degree=10)
The density of the graph is: 0.7


In [37]:
for node in subgt.nodes():
    subgt[node]['id'] = nodeLabels[str(node)]

Camellia_sinensis_leaf


In [39]:
attrs = {}
for node in subgt.nodes():
    attrs[node] = {'id': nodeLabels[str(node)]}
nx.set_node_attributes(subgt, attrs)

In [20]:
##get all neighbors of nodes
gtneighs = []
gtnodes = [
    obodict['greentea'][0],
    obodict['EGCG'][0],
    obodict['EGCG'][1],
    obodict['ECG'][0],
    obodict['epicatechin'][0],
    obodict['catechin'][0],
    obodict['catechin'][1],
    obodict['epigallocatechin'][0],
    obodict['gallocatechin'][0],
    obodict['gallocatechin'][1]
]
for node in gtnodes:
    neighs = [n for n in nx_graph.neighbors(node)]
    for item in neighs:
        gtneighs.append(item)
len(gtneighs)

3176

In [27]:
len(list(set(gtnodes)))

1787

In [23]:
subgt2 = nx_graph.subgraph(gtnodes)
get_graph_stats(subgt2)

There are 1787 nodes, 79081 edges, and 142 self-loop(s)
The Average Degree is 44.25349748181309
Nodes with highest degree:
Label: catechin
http://purl.obolibrary.org/obo/CHEBI_23053 (degree=1358)
Label: (-)-epigallocatechin gallate
http://napdi.org/napdi_srs_imports:epigallocatechin_gallate (degree=1257)
Label: quercetin
http://purl.obolibrary.org/obo/CHEBI_16243 (degree=1060)
Label: glucose
http://purl.obolibrary.org/obo/CHEBI_17234 (degree=980)
Label: (-)-epigallocatechin 3-gallate
http://purl.obolibrary.org/obo/CHEBI_4806 (degree=922)
Label: polyphenol
http://purl.obolibrary.org/obo/CHEBI_26195 (degree=918)
The density of the graph is: 0.024777994110757613


In [44]:
##save subgraphs as gephi files
nx.write_gexf(subgt, KG_PATH+'subgraphs/greentea_subgraph1.gexf')

TypeError: attribute value type is not allowed: <class 'rdflib.term.URIRef'>

In [ ]:
nx.write_gexf(subgt2, KG_PATH+'subgraphs/greentea_subgraph2.gexf')

In [ ]:
###merge the nt versions of graph and filter by nodes and edges? 